<a href="https://colab.research.google.com/github/Samanyu-16/Health_Chatbot/blob/main/Disease_Awareness_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== AUTO-FIX FOR NUMPY INCOMPATIBILITY (run first, every time) =====
import os, sys, importlib.util, time, site

# Check numpy version *before* import
try:
    import numpy as np
    version = tuple(map(int, np.__version__.split(".")[:2]))
    if version >= (2, 0):
        print(f"⚠️ Detected incompatible NumPy {np.__version__}. Reinstalling 1.26.4 ...")
        os.system("pip install -q --force-reinstall numpy==1.26.4")
        print("✅ NumPy downgraded to 1.26.4 — please restart runtime (Ctrl+M+.) once.")
        # Auto restart runtime (Colab restarts Python process)
        os.kill(os.getpid(), 9)
    else:
        print(f"✅ NumPy version {np.__version__} is compatible.")
except ModuleNotFoundError:
    print("Installing NumPy 1.26.4 ...")
    os.system("pip install -q numpy==1.26.4")
    os.kill(os.getpid(), 9)


✅ NumPy version 1.26.4 is compatible.


In [ ]:
# Preflight: fully uninstall packages that cause ABI problems, then install numpy 1.26.4
# Run this FIRST in the fresh session.

!pip -q uninstall -y faiss-cpu faiss-gpu opencv-python opencv-python-headless opencv-contrib-python numba || true
!pip -q install --no-cache-dir --upgrade --force-reinstall "numpy==1.26.4"

# verify
import sys, numpy as np
print("Python:", sys.version.splitlines()[0])
print("NumPy:", np.__version__)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 201.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
umap-learn 0.5.9.post2 requires numba>=0.51.2, which is not installed.
shap 0.50.0 requires numba>=0.54; python_version < "3.14", which is not installed.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.
cuml-cu12 25.10.0 requires numba<0.62.0a0,>=0.60.0, which is not installed.
pynndescent 0.5.13 requires numba>=0.51.2, which is not installed.
stumpy 1.13.0 requires numba>=0.57.1, which is not installed.
cudf-cu12 25.10.0 requires numba<0.62.0a0,>=0.60.0, which is not installed.
librosa 0.11.0 requires numba>=0.51.0, which is not installed.
albumen

In [ ]:
# Install model & UI libs after NumPy pinned.
!pip -q install --no-cache-dir "gradio==4.19.1" transformers duckduckgo-search wikipedia googletrans==4.0.0-rc1 requests beautifulsoup4 lxml



  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 277.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 198.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 216.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 158.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 217.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 198.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 269.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 251.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 186.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ===== FINAL: Robust Multilingual Voice+Text Health Chatbot (single cell) =====
# Paste & run after your numpy/ffmpeg preflight cells.

!apt-get update -qq
!apt-get install -y -qq ffmpeg
!pip -q install --no-cache-dir transformers gradio wikipedia googletrans==4.0.0-rc1 SpeechRecognition pydub gTTS duckduckgo-search rapidfuzz requests

import os, time, uuid, json, traceback
from pathlib import Path
from datetime import datetime, timezone
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from googletrans import Translator
from gtts import gTTS
from pydub import AudioSegment
import speech_recognition as sr
from rapidfuzz import process, fuzz
import wikipedia as wiki
from duckduckgo_search import DDGS

# ---------- Config ----------
MODEL_NAME = "google/flan-t5-large"   # change to flan-t5-large if you have GPU+RAM
CACHE_DIR = Path("/content/disease_cache"); CACHE_DIR.mkdir(parents=True, exist_ok=True)
LOG_PATH = "/content/healthbot_logs.csv"
DISCLAIMER = "⚕️ I'm an AI health assistant — not a doctor. For medical advice, consult a qualified professional."

# small index (expand with local synonyms for better matching)
ICD_TINY = ["dengue", "malaria", "covid-19", "influenza", "tuberculosis", "hepatitis a", "hepatitis b",
            "hepatitis c", "measles", "pneumonia", "typhoid", "cholera", "rabies", "zika", "chikungunya", "leukemia"]
ICD_TERMS = ICD_TINY.copy()

# ---------- Device & Model ----------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE, "Loading model (may take a while)...")
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
gen = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(DEVICE)
print("Model loaded:", MODEL_NAME)

# ---------- Translator & wiki ----------
translator = Translator()
wiki.set_lang("en")

# ---------- Offline DB ----------
OFFLINE_DB = {
    "dengue": {"Overview":"Dengue is a mosquito-borne viral infection common in tropical regions.","Symptoms":"High fever, severe headache, muscle and joint pain, nausea, rash.","Causes":"Dengue virus transmitted by Aedes mosquitoes.","Prevention":"Avoid mosquito bites, remove standing water, use repellents.","Treatment":"Supportive care; seek medical help for warning signs.","Complications":"Severe dengue can cause bleeding and shock."},
    "malaria": {"Overview":"Malaria is a parasitic disease transmitted by mosquitoes.","Symptoms":"Fever, chills, sweats, headache, vomiting, fatigue.","Causes":"Plasmodium parasites transmitted by mosquito bites.","Prevention":"Use bed nets and prophylaxis where recommended.","Treatment":"Antimalarial drugs prescribed by doctors."},
    "leukemia": {"Overview":"Leukemia is a cancer affecting blood-forming tissues.","Symptoms":"Fatigue, fever, frequent infections, easy bruising or bleeding.","Causes":"Varies by type; includes genetic and environmental factors.","Prevention":"No guaranteed prevention; routine checks if at risk.","Treatment":"Chemotherapy, targeted therapy, bone marrow transplant.","Complications":"Infections, anemia, bleeding, organ dysfunction."}
}

# ---------- Logging ----------
def ensure_log_header():
    if not os.path.exists(LOG_PATH):
        with open(LOG_PATH, "w", newline="", encoding="utf-8") as f:
            f.write("timestamp,session_id,lang,mode,query,response,error\n")
def log_event(session_id, lang, mode, query, response, error=""):
    ensure_log_header()
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(",".join([datetime.now(timezone.utc).isoformat(), session_id, str(lang), str(mode),
                          json.dumps(query).replace(',', ';'), json.dumps(response).replace(',', ';'), json.dumps(error).replace(',', ';')]) + "\n")

# ---------- Fuzzy ----------
def fuzzy_match_term(q, score_cutoff=55):
    if not ICD_TERMS: return None, 0
    match = process.extractOne(q, ICD_TERMS, scorer=fuzz.WRatio, score_cutoff=score_cutoff)
    if match:
        term, score, _ = match
        return term, int(score)
    return None, 0

# ---------- Generation (structured & long) ----------
def generate_structured_entry(term_or_query, extra_context=""):
    prompt = (
        "You are an experienced public-health educator creating patient-facing content. "
        "Write clear, evidence-based explanations and organize the answer into labeled sections. "
        "Each labeled section should be 3-5 complete sentences.\n\n"
        f"Topic: {term_or_query}\n\n"
        "Sections (include exactly these labels):\n"
        "Overview:\nSymptoms:\nCauses:\nPrevention:\nTreatment:\nComplications:\n\n"
        f"Context: {extra_context}\n\n"
        "End with: ⚕️ Disclaimer: Consult a healthcare professional for personal advice."
    )
    try:
        inp = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to(DEVICE)
        out = gen.generate(
            **inp,
            max_new_tokens=800,
            temperature=0.18,
            top_p=0.95,
            num_beams=6,
            no_repeat_ngram_size=3,
            early_stopping=True,
            length_penalty=0.8
        )
        txt = tok.decode(out[0], skip_special_tokens=True)
        return txt.strip()
    except Exception as e:
        print("Generation failed:", e)
        traceback.print_exc()
        return ""

# ---------- Caching ----------
def cache_path_for_term(term):
    safe = term.replace(" ", "_").replace("/", "_").lower()
    return CACHE_DIR / f"{safe}.json"
def get_cached_entry(term):
    p = cache_path_for_term(term)
    if p.exists():
        try:
            return json.loads(p.read_text(encoding="utf-8"))["entry"]
        except:
            return None
    return None
def save_cached_entry(term, text):
    p = cache_path_for_term(term)
    with open(p, "w", encoding="utf-8") as f:
        json.dump({"term": term, "entry": text, "ts": time.time()}, f, ensure_ascii=False, indent=2)

# ---------- TTS (chunk) ----------
def text_to_speech_mp3(long_text, lang_code):
    try:
        if not long_text or long_text.strip() == "": return None
        max_chars = 1400
        parts = [long_text[i:i+max_chars] for i in range(0, len(long_text), max_chars)]
        audio_segments, tmp_files = [], []
        for i, part in enumerate(parts):
            tts = gTTS(text=part, lang=lang_code if lang_code else "en")
            out_file = f"/content/tts_{uuid.uuid4().hex}_{i}.mp3"
            tts.save(out_file)
            tmp_files.append(out_file)
            seg = AudioSegment.from_file(out_file)
            audio_segments.append(seg)
        combined = audio_segments[0]
        for seg in audio_segments[1:]:
            combined += seg
        final = f"/content/voice_reply_{uuid.uuid4().hex}.mp3"
        combined.export(final, format="mp3")
        for tf in tmp_files:
            try: os.remove(tf)
            except: pass
        return final
    except Exception as e:
        print("TTS failed:", e)
        traceback.print_exc()
        return None

# ---------- STT improved ----------
recognizer = sr.Recognizer()
_STT_CANDIDATES = [("kn-IN","kn"), ("hi-IN","hi"), ("ta-IN","ta"), ("te-IN","te"), ("en-US","en")]

def speech_to_text_best(audio_path, preferred_lang=None, debug=False):
    """
    Multi-pass STT: tries several languages, uses show_all alternatives, scores candidates.
    Returns (best_text, best_lang) or with debug True: (best_text, best_lang, debug_info)
    """
    transcripts = []
    wav_path = f"/content/speech_{uuid.uuid4().hex}.wav"
    try:
        AudioSegment.from_file(audio_path).export(wav_path, format="wav")
    except Exception as e:
        if debug: return "", "en", {"error":"convert_failed", "exception": str(e)}
        return "", "en"

    candidates = _STT_CANDIDATES.copy()
    if preferred_lang:
        mapping = {c[1]: c for c in candidates}
        if preferred_lang in mapping:
            cand = mapping[preferred_lang]
            candidates = [cand] + [c for c in candidates if c != cand]

    try:
        with sr.AudioFile(wav_path) as src:
            recognizer.adjust_for_ambient_noise(src, duration=0.4)
            raw = recognizer.record(src)
    except Exception as e:
        if debug: return "", "en", {"error":"audio_read_failed", "exception": str(e)}
        return "", "en"

    for stt_lang_code, short_code in candidates:
        try:
            res = recognizer.recognize_google(raw, language=stt_lang_code, show_all=True)
            alts = []
            if isinstance(res, dict) and "alternative" in res:
                for a in res["alternative"]:
                    text = a.get("transcript","").strip()
                    conf = a.get("confidence", 0.0)
                    if text: alts.append({"text": text, "confidence": conf})
            elif isinstance(res, list):
                for r in res:
                    if "transcript" in r: alts.append({"text": r.get("transcript","").strip(), "confidence": r.get("confidence", 0.0)})
            elif isinstance(res, str):
                alts.append({"text": res, "confidence": 0.0})
            if not alts:
                alts = [{"text":"", "confidence": 0.0}]
            for alt in alts:
                transcripts.append({"lang": short_code, "stt_lang_code": stt_lang_code, "text": alt["text"], "stt_confidence": alt.get("confidence",0.0)})
        except Exception as e:
            transcripts.append({"lang": short_code, "stt_lang_code": stt_lang_code, "text": "", "error": str(e)})

    scored = []
    for t in transcripts:
        txt = (t.get("text") or "").strip()
        if not txt:
            scored.append({**t, "score":0, "match_term":None, "text_en":""})
            continue
        try:
            txt_en = translator.translate(txt, src=t["lang"], dest="en").text if t["lang"] != "en" else txt
        except Exception:
            try: txt_en = translator.translate(txt, dest="en").text
            except: txt_en = txt
        best = process.extractOne(txt_en, ICD_TERMS, scorer=fuzz.WRatio) if ICD_TERMS else None
        score = int(best[1]) if best else 0
        match_term = best[0] if best else None
        db_score = 0; db_match = None
        for k in OFFLINE_DB.keys():
            s = fuzz.WRatio(txt_en.lower(), k.lower())
            if s > db_score:
                db_score = int(s); db_match = k
        if db_score > score:
            score = db_score; match_term = db_match
        stt_conf = t.get("stt_confidence", 0.0)
        combined = int(score * 0.85 + stt_conf * 15)
        scored.append({**t, "score": combined, "match_term": match_term, "text_en": txt_en})

    best_item = max(scored, key=lambda x: x.get("score",0)) if scored else {"text":"", "lang":"en", "score":0}
    if best_item.get("score",0) < 50:
        alt_by_conf = sorted([s for s in scored if s.get("text")], key=lambda x: x.get("stt_confidence",0), reverse=True)
        if alt_by_conf:
            best_item = alt_by_conf[0]
        else:
            fallback = next((x for x in scored if x.get("text")), None)
            if fallback: best_item = fallback

    debug_info = {"candidates": scored}
    if debug:
        return best_item.get("text",""), best_item.get("lang","en"), debug_info
    return best_item.get("text",""), best_item.get("lang","en")

# ---------- Web context helpers ----------
def build_context_for_query(q):
    ctx_parts = []
    try:
        s = wiki.search(q, results=1)
        if s:
            try: ctx_parts.append(wiki.summary(s[0], sentences=3))
            except: pass
    except: pass
    try:
        with DDGS() as ddgs:
            for r in ddgs.text(f"{q} site:who.int OR site:cdc.gov OR site:mayoclinic.org", max_results=2):
                href = r.get("href","")
                if href: ctx_parts.append(f"Reference: {href}")
    except: pass
    return "\n".join(ctx_parts)

# ---------- Main pipeline ----------
def process_query(user_text, lang_choice, mode, session_id):
    try:
        detected = translator.detect(user_text).lang if user_text.strip() else "en"
    except: detected = "en"
    target_lang = None if lang_choice == "Auto-detect" else (lang_choice if lang_choice in ["en","hi","kn","ta","te"] else None)
    if target_lang is None: target_lang = detected or "en"
    q_en = user_text
    if detected != "en":
        try: q_en = translator.translate(user_text, src=detected, dest="en").text
        except: q_en = user_text
    EMERGENCY_TRIGGERS = ["severe chest pain","shortness of breath","unconscious","stroke","seizure","fainting","suicidal","heavy bleeding","difficulty breathing"]
    if any(k in q_en.lower() for k in EMERGENCY_TRIGGERS):
        em = ("🚨 Emergency detected. Please call local emergency services immediately.\n\n" + DISCLAIMER)
        audio = text_to_speech_mp3(em, target_lang)
        log_event(session_id, target_lang, mode, user_text, em, error="emergency")
        return em, audio

    entry_text = None
    matched_term, score = fuzzy_match_term(q_en, score_cutoff=50)
    if matched_term:
        entry_text = get_cached_entry(matched_term)
        if not entry_text:
            ctx = build_context_for_query(q_en)
            generated = generate_structured_entry(matched_term, extra_context=ctx)
            if generated and len(generated.split())>10:
                entry_text = generated
                save_cached_entry(matched_term, entry_text)

    if not entry_text:
        ctx = build_context_for_query(q_en)
        generated = generate_structured_entry(q_en, extra_context=ctx)
        if generated and len(generated.split())>10:
            entry_text = generated

    if not entry_text:
        found = None
        for name, data in OFFLINE_DB.items():
            if name in q_en.lower() or any(token in q_en.lower() for token in name.split()):
                parts=[]
                for sec in ["Overview","Symptoms","Causes","Prevention","Treatment","Complications"]:
                    if sec in data: parts.append(f"{sec}: {data[sec]}")
                parts.append(DISCLAIMER)
                found = "\n\n".join(parts)
                break
        if found: entry_text = found
        else:
            entry_text = ("Sorry — I couldn't find detailed public health info for that exact query. "
                          "Try a simpler disease name (e.g., 'dengue') or ask in text mode. \n\n" + DISCLAIMER)

    final_text = entry_text
    if target_lang and target_lang != "en":
        try: final_text = translator.translate(entry_text, src="en", dest=target_lang).text
        except Exception as e: print("translate_out failed:", e); final_text = entry_text

    audio_path = text_to_speech_mp3(final_text, target_lang or "en")
    log_event(session_id, target_lang, mode, user_text, final_text)
    return final_text, audio_path

# ---------- Gradio UI ----------
css = """
#title { text-align:center; padding:12px; background:linear-gradient(135deg,#7b2ff7,#f107a3); color:white; border-radius:8px; }
#footer { text-align:center; color:#aaa; padding-top:6px;}
"""
with gr.Blocks(css=css, theme=gr.themes.Soft()) as demo:
    gr.HTML('<div id="title"><h2>🌍 Multilingual Health Assistant (Voice + Text)</h2><p>Pick the Reply language (matching your speech) before recording for best results.</p></div>')
    session_id = gr.State(str(uuid.uuid4()))
    with gr.Row():
        lang_choice = gr.Dropdown(choices=["Auto-detect","en","hi","ta","te","kn"], value="Auto-detect", label="Reply language (choose or Auto-detect)")
        mode = gr.Radio(choices=["Auto","Disease Awareness"], value="Disease Awareness", label="Mode")

    # Text Chat
    with gr.Tab("Text Chat"):
        text_in = gr.Textbox(placeholder="e.g., What are the symptoms of dengue?", label="Type your health question")
        text_btn = gr.Button("Submit Text Question")
        text_out = gr.Markdown()
        audio_out = gr.Audio(label="Voice Reply (text mode)")
        text_btn.click(fn=lambda q,lc,m,s: process_query(q,lc,m,s)[0], inputs=[text_in, lang_choice, mode, session_id], outputs=text_out)
        text_btn.click(fn=lambda q,lc,m,s: process_query(q,lc,m,s)[1], inputs=[text_in, lang_choice, mode, session_id], outputs=audio_out)

       # --- Voice Chat (no debug shown) ---
       # --- Voice Chat: FORCE selected language for STT (Option 1) ---
    with gr.Tab("Voice Chat"):
        mic = gr.Audio(type="filepath", label="Record your voice (works on mobile & desktop browsers)")
        voice_btn = gr.Button("Submit Voice Question")
        voice_text = gr.Markdown()
        voice_audio = gr.Audio(label="Voice Reply (voice mode)")

        # Option 1 handler: force STT to use only the selected language (if not Auto-detect)
        def voice_handler_force_lang(audio, lc, m, s):
            if not audio:
                return "No audio uploaded.", None

            # If user explicitly chose a language, force STT to use only that language first.
            if lc and lc != "Auto-detect":
                # Map UI code (e.g., "kn") to a stt candidate code list
                mapping = {
                    "kn": [("kn-IN","kn")],
                    "hi": [("hi-IN","hi")],
                    "ta": [("ta-IN","ta")],
                    "te": [("te-IN","te")],
                    "en": [("en-US","en")]
                }
                candidates_override = mapping.get(lc, None)
            else:
                candidates_override = None

            try:
                # if we have an override, ask STT to use only those candidates
                if candidates_override:
                    # speech_to_text_best must accept candidates_override (see earlier patch)
                    transcript, detected_lang = speech_to_text_best(audio, preferred_lang=lc, debug=False, candidates_override=candidates_override)
                else:
                    transcript, detected_lang = speech_to_text_best(audio, preferred_lang=None, debug=False)
            except Exception as e:
                print("speech_to_text_best failed:", e)
                transcript, detected_lang = "", "en"

            if not transcript:
                return "Sorry, couldn't transcribe your audio. Please try again or switch to text mode.", None

            displayed_intro = f"**Transcribed ({detected_lang}):** {transcript}\n\n---\n\n**Assistant reply:**\n\n"
            try:
                resp_text, resp_audio = process_query(transcript, lc, m, s)
            except Exception as e:
                print("process_query failed:", e, traceback.format_exc())
                return "Sorry, an internal error occurred while processing your question.", None
            combined_text = displayed_intro + resp_text
            return combined_text, resp_audio

        # Wire the button to the new forced-language handler
        voice_btn.click(voice_handler_force_lang, inputs=[mic, lang_choice, mode, session_id], outputs=[voice_text, voice_audio])



    gr.File(label="📥 Download Logs", value=lambda: LOG_PATH if os.path.exists(LOG_PATH) else None, interactive=True)
    gr.HTML('<div id="footer">⚕️ For educational purposes only. Not a substitute for professional medical advice.</div>')

demo.launch(share=True)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 210.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 215.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml-cu12 25.10.0 requires numba<0.62.0a0,>=0.60.0, which is not installed.
mcp 1.21.0 requires httpx>=0.27.1, but you have httpx 0.13.3 which is incompatible.
google-adk 1.17.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 11.0.3 which is incompatible.
Device: cuda Loading model (may take a while)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded: google/flan-t5-large
IMPORTANT: You are using gradio version 4.19.1, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c4e9a3385b065cd292.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
